In [1]:
import requests
from bs4 import BeautifulSoup
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(URL) 
soup = BeautifulSoup(req.content, 'html5lib')  
table = soup.find('tbody')
li_ad=[]
for row in table.find_all('tr'): 
    neigh = {}
    cell = row.find_all('td')
    if cell:
        neigh['Postal_Code']=cell[0].get_text()
        neigh['Borough']=cell[1].get_text()
        neigh['Neighborhood']=cell[2].get_text()
        li_ad.append(neigh)

import pandas as pd
import numpy as np
neighbours = pd.DataFrame(li_ad)
neighbours = neighbours.replace('\n','', regex=True)
neighbours=neighbours[neighbours.Borough != 'Not assigned']
print(neighbours.shape)
neighbours.head()

(103, 3)


,Postal_Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [2]:
!pip3 install geocoder
#didnot work. only returning None

     |████████████████████████████████| 98 kB 205 kB/s eta 0:00:01
     |████████████████████████████████| 829 kB 1.2 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=0cd6c96f9349f62c30c5a2105920e7e4824e26287b365ace9f7e223f7ac6446d
  Stored in directory: /home/sid/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future


In [20]:
import geocoder # import geocoder
neighbours['latitude'] = 'nan'
neighbours['longitude'] = 'nan'
geo_code=pd.read_csv('can_data.csv')
geo_code.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
for ind in neighbours.index:    
    # initialize your variable to None
    for jnd in geo_code.index:
        if neighbours['Postal_Code'][ind] == geo_code['Postal_Code'][jnd]:
            neighbours['latitude'][ind] = geo_code['Latitude'][jnd]    
            neighbours['longitude'][ind] = geo_code['Longitude'][jnd]

neighbours.head()

,Postal_Code,Borough,Neighborhood,latitude,longitude
2,M3A,North York,Parkwoods,43.7533,-79.3297
3,M4A,North York,Victoria Village,43.7259,-79.3156
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


In [37]:
toro = neighbours[neighbours['Borough'].str.contains("Toronto")]
print(toro.shape)
toro.head()


(39, 5)


,Postal_Code,Borough,Neighborhood,latitude,longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
22,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
30,M4E,East Toronto,The Beaches,43.6764,-79.293


In [43]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toro_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [45]:
import folium
map_toro = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toro['latitude'], toro['longitude'], toro['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toro)  
    
map_toro

In [52]:
LIMIT=200
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
toro_venues = getNearbyVenues(names=toro['Neighborhood'],
                                   latitudes=toro['latitude'],
                                   longitudes=toro['longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [55]:
toro_venues.shape

(1612, 7)

In [56]:
toro_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [61]:
# one hot encoding
toro_encodes = pd.get_dummies(toro_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toro_encodes['Neighborhood'] = toro_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toro_encodes.columns[-1]] + list(toro_encodes.columns[:-1])
toro_encodes = toro_encodes[fixed_columns]

toro_encodes.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
toro_encode.shape

(1612, 240)

In [64]:
toro_grouped = toro_encodes.groupby('Neighborhood').mean().reset_index()
toro_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,Central Bay Street,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.015625,0.000000,0.000000,0.015625,0.000000,0.000000,0.00000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
6,Church and Wellesley,0.025974,0.012987,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.012987,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.00000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.00000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.027778,...,0.00000,0.00,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [65]:
num_top_venues = 5

for hood in toro_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toro_grouped[toro_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3       Nightclub  0.09
4             Bar  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1          Smoke Shop  0.06
2             Brewery  0.06
3       Auto Workshop  0.06
4              Garden  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2   Harbor / Marina  0.07
3  Sculpture Garden  0.07
4       Coffee Shop  0.07


----Central Bay Street----
                venue  freq
0         Coffee Sho

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toro_grouped['Neighborhood']

for ind in np.arange(toro_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toro_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Café,Gourmet Shop,Japanese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Breakfast Spot,Coffee Shop,Performing Arts Venue,Restaurant,Bakery,Stadium,Intersection,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Pizza Place,Restaurant,Fast Food Restaurant,Farmers Market,Skate Park,Auto Workshop,Burrito Place,Spa,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Coffee Shop,Airport Terminal,Plane,Harbor / Marina,Bar,Boat or Ferry,Sculpture Garden,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Ice Cream Shop,Japanese Restaurant,Salad Place,Thai Restaurant,Bubble Tea Shop,Burger Joint


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)